In [1]:
import pandas as pd
from sqlalchemy import create_engine
from getpass import getpass
from urllib.parse import quote_plus

pd.set_option('display.max_columns', None)

In [2]:
host = 'aws-1-us-east-1.pooler.supabase.com'.strip()
port = '5432'.strip()
database = 'postgres'.strip()
user = 'candidato.jdazostyahhxukbmxybw'.strip()
password = 'DesafioInov@2026!'
password_cod = quote_plus(password)

connection_string = f"postgresql+psycopg2://{user}:{password_cod}@{host}:{port}/{database}"

try:
    engine = create_engine(connection_string)
    with engine.connect() as connection:
        print("Conexão bem-sucedida!")
except Exception as e:
    print(f"Erro ao conectar: {e}")

Conexão bem-sucedida!


In [84]:
'''
Questão 1: Existem pagamentos e liquidações sem um empenho associado?
'''

query_pag1 = """
SELECT * FROM pagamento WHERE pagamento.id_empenho IS NULL
"""

query_liq1 = """
SELECT * FROM liquidacao_nota_fiscal WHERE liquidacao_nota_fiscal.id_empenho IS NULL
"""

resultpag1 = pd.read_sql(query_pag1, engine)
resultliq1 = pd.read_sql(query_liq1, engine)
                        
print(f"Quantidade de pagamentos sem empenhos é de {len(resultpag1)}");
display(resultpag1)

print(f"Quantidade de liquidações sem empenhos é de {len(resultliq1)}");
display(resultliq1)



Quantidade de pagamentos sem empenhos é de 0


,id_empenho,id_pagamento,datapagamentoempenho,valor


Quantidade de liquidações sem empenhos é de 0


,id_liquidacao_empenhonotafiscal,chave_danfe,data_emissao,valor,id_empenho


In [5]:
'''
Questão 2: Há algum empenho que não esteja associado a nenhum contrato? 
'''

query_emp2 = """
SELECT * FROM empenho WHERE empenho.id_contrato IS NULL
"""
result = pd.read_sql(query_emp2, engine)
print(f"Número de empenhos que não possuem contratos associados: {len(result)}")
display(result)

Número de empenhos que não possuem contratos associados: 0


,id_empenho,ano,data_empenho,cpfcnpjcredor,credor,valor,id_entidade,id_contrato


In [46]:
'''
Questão 3: Há algum contrato que não possui entidade ou fornecedor?
'''

query_ent3 = """
SELECT * FROM contrato WHERE contrato.id_entidade IS NULL
"""

query_for3 = """
SELECT * FROM contrato WHERE contrato.id_fornecedor IS NULL
"""

resultent3 = pd.read_sql(query_ent3, engine)
resultfor3 = pd.read_sql(query_for3, engine)

print(f"Número de contratos que não possuem entidades associadas:{len(resultent3)}")
display(resultent3)

print(f"Número de contratos que não possuem fornecedores associados: {len(resultfor3)}")
display(resultfor3)

Número de contratos que não possuem entidades associadas:0


,id_contrato,valor,data,objeto,id_entidade,id_fornecedor


Número de contratos que não possuem fornecedores associados: 0


,id_contrato,valor,data,objeto,id_entidade,id_fornecedor


In [3]:
'''
Questão 4: Há algum registro no qual o valor do empenho seja superior que o valor do contrato?
'''

query_reg4 = """
SELECT 
    e.id_empenho, 
    e.credor, 
    e.valor as valor_empenho,
    e.data_empenho,

    c.id_contrato,
    c.valor as valor_contrato,
    c.data as data_contrato

FROM contrato c INNER JOIN empenho e ON c.id_contrato = e.id_contrato
"""

result = pd.read_sql(query_reg4, engine)

empenho_superior = result[result['valor_empenho']>result['valor_contrato']]
empenho_superior['diferenca'] = empenho_superior['valor_empenho'] - result['valor_contrato']

print(f"O número de empenhos com valores maiores do que o do contrato é de: {len(empenho_superior)}")
display(empenho_superior.sort_values('diferenca', ascending=False))

resultadinho = pd.read_sql('SELECT * FROM empenho e INNER JOIN contrato c ON c.id_contrato = e.id_contrato', engine)

resultadinho.to_csv('empenhos.csv', index=False, sep=';', decimal=',')

O número de empenhos com valores maiores do que o do contrato é de: 150


,id_empenho,credor,valor_empenho,data_empenho,id_contrato,valor_contrato,data_contrato,diferenca
359,EMP-11,Saude Vida Distribuidora,82476.26,2024-01-14,1013,49985.61,2024-01-02,32490.65
480,EMP-224,TechServer Solucoes,82166.98,2024-01-08,1139,49798.17,2024-01-11,32368.81
388,EMP-140,Transportes Rapido,81629.94,2024-01-12,1046,49472.69,2024-01-10,32157.25
371,EMP-122,LimpMax Servicos,80642.35,2024-01-15,1026,48874.15,2024-01-06,31768.20
487,EMP-231,Merenda Boa Alimentos,80564.57,2024-01-07,1147,48827.01,2024-01-07,31737.56
...,...,...,...,...,...,...,...,...
463,EMP-215,Eventos & Promo,3948.37,2024-01-07,1129,2392.95,2024-01-02,1555.42
445,EMP-199,ConstruRio Engenharia,3266.16,2024-01-12,1110,1979.49,2024-01-01,1286.67
481,EMP-225,ConstruRio Engenharia,2519.62,2024-01-08,1140,1527.04,2024-01-11,992.58
442,EMP-196,Saude Vida Distribuidora,2502.92,2024-01-13,1107,1516.92,2024-01-10,986.00


In [88]:
'''
Questão 5: Existem registros com valor liquidado maior que o valor empenhado?
'''

query_liq5 = """
SELECT 
    e.id_empenho,
    e.valor as valor_empenho,
    e.credor,
    e.data_empenho,
    l.id_liquidacao_empenhonotafiscal,
    l.valor as valor_liquidado,
    l.data_emissao

FROM empenho e INNER JOIN liquidacao_nota_fiscal l ON e.id_empenho = l.id_empenho
"""

resultliq5 = pd.read_sql(query_liq5, engine)

liq_superior = resultliq5[resultliq5['valor_liquidado']>resultliq5['valor_empenho']]

liq_superior['diferenca'] = liq_superior['valor_liquidado'] - resultliq5['valor_empenho']

print(f"O número de registros que possuem valor liquidado maior que valor empenhado é de: {len(liq_superior)}")
display(liq_superior.sort_values('diferenca', ascending=False))

liq_superior.to_csv('anomalia_liquidacao.csv', index=False, sep=";", decimal=',')

O número de registros que possuem valor liquidado maior que valor empenhado é de: 211


,id_empenho,valor_empenho,credor,data_empenho,id_liquidacao_empenhonotafiscal,valor_liquidado,data_emissao,diferenca
395,EMP-430,1024.70,Obras Viarias RJ,2024-01-06,5430,45603.04,2024-01-14,44578.34
255,EMP-277,2261.26,Papelaria Central,2024-01-12,5277,46520.70,2024-01-27,44259.44
264,EMP-286,6451.56,ConstruRio Engenharia,2024-01-05,5286,49633.66,2024-01-11,43182.10
449,EMP-492,6657.12,LimpMax Servicos,2024-01-03,5492,49394.11,2024-01-11,42736.99
391,EMP-426,6557.90,Consultoria Estrategica,2024-01-08,5426,49232.11,2024-01-24,42674.21
...,...,...,...,...,...,...,...,...
87,EMP-93,25340.64,TI Inovacao,2024-01-18,5093,25764.12,2024-01-30,423.48
17,EMP-19,17109.77,Eventos & Promo,2024-01-05,5019,17427.94,2024-01-20,318.17
221,EMP-240,49051.89,Merenda Boa Alimentos,2024-01-04,5240,49220.70,2024-01-16,168.81
156,EMP-168,8884.69,Saude Vida Distribuidora,2024-01-13,5168,8928.78,2024-01-21,44.09


In [90]:
'''
Questão 6: Existe algum registro no qual a data do pagamento ocorreu antes da data da liquidação?
'''

query_pag6 = """
SELECT
    p.id_pagamento,
    l.data_emissao as data_liquidacao,
    p.datapagamentoempenho as data_pagamento,
    e.credor,
    (l.data_emissao - p.datapagamentoempenho) as dias_antecipados
FROM pagamento p INNER JOIN empenho e ON p.id_empenho = e.id_empenho
INNER JOIN liquidacao_nota_fiscal l ON l.id_empenho = e.id_empenho
WHERE p.datapagamentoempenho < l.data_emissao
"""


resultpag6 = pd.read_sql(query_pag6, engine)

print(f"Número de registros que possuem a data de pagamento anterior a data de liquidaçao: {len(resultpag6)}")
display(resultpag6.sort_values('dias_antecipados', ascending=False))

resultpag6.to_csv('anomalia_temporal.csv', index=False, sep=';', decimal=',')




Número de registros que possuem a data de pagamento anterior a data de liquidaçao: 41


,id_pagamento,data_liquidacao,data_pagamento,credor,dias_antecipados
16,PGT-46,2024-02-08,2024-01-14,Papelaria Central,25
25,PGT-76,2024-02-04,2024-01-10,Consultoria Estrategica,25
19,PGT-484,2024-01-31,2024-01-06,Saude Vida Distribuidora,25
14,PGT-450,2024-02-01,2024-01-07,Merenda Boa Alimentos,25
36,PGT-202,2024-02-04,2024-01-10,Merenda Boa Alimentos,25
28,PGT-10,2024-02-01,2024-01-08,ConstruRio Engenharia,24
27,PGT-83,2024-02-08,2024-01-15,LimpMax Servicos,24
13,PGT-446,2024-01-25,2024-01-02,Transportes Rapido,23
0,PGT-244,2024-01-26,2024-01-03,TI Inovacao,23
24,PGT-74,2024-01-22,2023-12-31,Saude Vida Distribuidora,22


In [76]:
'''
Questão 7: Existem liquidações feitas antes de algum empenho?
'''

query_liq7 = """
SELECT
    e.id_empenho,
    e.data_empenho,
    l.data_emissao as emissao_liquidacao,
    (e.data_empenho - l.data_emissao) as dias_antecipados
FROM empenho e INNER JOIN liquidacao_nota_fiscal l ON l.id_empenho = e.id_empenho
WHERE l.data_emissao < e.data_empenho
"""

resultliq7 = pd.read_sql(query_liq7, engine)
print(f"Número de registros onde as liquidações ocorreram antes do empenho: {len(resultliq7)}")
display(resultliq7.sort_values('dias_antecipados', ascending=False))

Número de registros onde as liquidações ocorreram antes do empenho: 0


,id_empenho,data_empenho,emissao_liquidacao,dias_antecipados
